# The Sed object

A Spectral Energy Distribution, or SED, describes the energy distribution of an emitting body as a function of frequency / wavelength. In Synthesizer, spectra are stored in ``Sed`` objects. 

There are a number of different ways to generate spectra in Synthesizer, but in every case the resulting SED is always stored in an ``Sed`` object. 
An ``Sed`` object in Synthesizer is generally agnostic to where the input spectra has come from, as you'll see below; they can therefore be initialised with any arbitrary wavelength and luminosity density arrays.

The ``Sed`` class contains a number of methods for calculating derived properties, such as broadband luminosities / fluxes (within wavelength windows or on [filters](../../observatories/filters.rst)) and spectral indices (e.g. balmer break, UV-continuum slope), and for modifying the spectra themselves, for example by applying an attenuation curve due to dust. Many of these methods are automatically used during emission generation with an ``EmissionModel``, but they can also be used directly from the ``Sed`` objects themselves. 

In this section we cover the basics of instantiating and interacting with ``Sed`` objects, and how to use them to calculate derived properties.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from unyt import Angstrom, Hz, erg, eV, s, um

## Initialising an Sed object

Although ``Sed`` objects are generally generated by other classes, you can also create them directly. The simplest way to do this is to pass a wavelength array and a luminosity density array. 


In [ ]:
from synthesizer import Sed

# Define some wavelengths and luminosities densities
lams = np.logspace(3, 5, 100) * Angstrom
lnus = np.logspace(26, 30, 100) * erg / (s * Hz)

# Create a Sed object
sed = Sed(lams, lnus)

## Extracting a Sed from a Grid

We cover generating ``Sed`` objects from galaxies and their components in the Generating Emissions section of the [emissions documentation](../emissions.rst). The simplest way to generate a real ``Sed`` object is to extract it from a ``Grid`` directly (see the [Grids documentation](../../emission_grids/grids.rst) for more details). First we need the ``Grid``.

In [ ]:
from synthesizer.grid import Grid

# Load a SPS grid
grid = Grid("test_grid")

Now we have a grid, we can extract an ``Sed`` for a single grid point by stating the age and metallicity of the point we want to extract, converting that to the spectra's index, and then extracting the spectra at that index. 

Note that we multiply the spectra below by an "initial mass" of $10^8 {M_\odot}$ because spectra in this ``Grid`` are in per unit initial mass, and we want the values for the rest of this section to make sense. This is mostly unimportant.

In [ ]:
log10age = 6.0  # log10(age/yr)
metallicity = 0.01
spectra_type = "incident"
grid_point = grid.get_grid_point(log10ages=log10age, metallicity=metallicity)
sed = grid.get_sed_at_grid_point(grid_point, spectra_type=spectra_type)
sed.lnu *= 1e8  # multiply initial stellar mass

## Sed attributes

Like all objects in Synthesizer, printing a ``Sed`` object prints a tabular summary of the object.

In [ ]:
print(sed)

The most important attributes of a ``Sed`` are `lam` and `lnu`, the wavelengths and spectral luminosity density respectively. 

In [ ]:
print(sed.lam)
print(sed.lnu)

These also have more descriptive aliases.

In [ ]:
print(sed.wavelength)
print(sed.luminosity_nu)

We can also get the luminosity ($L$) or spectral luminosity density per wavelength ($L_{\lambda}$).

In [ ]:
print(sed.luminosity)
print(sed.llam)

## Plotting a Sed 

We could just take these attributes and plot them directly.

In [ ]:
plt.plot(np.log10(sed.lam), np.log10(sed.lnu))
plt.show()

But ``Sed`` objects have their own bespoke plotting method, which is a little more sophisticated.

In [ ]:
fig, ax = sed.plot_spectra(label=f"log10(age/yr)={log10age}, Z={metallicity}")

We can also visualise the spectrum as a rainbow.

In [ ]:
fig, ax = sed.plot_spectra_as_rainbow()

## Arithmetic with Sed objects

``Sed`` objects can be easily scaled via the `*` operator. 




In [ ]:
sed_x_10 = sed * 10

fig, ax = sed.plot_spectra(label="original")
fig, ax = sed_x_10.plot_spectra(label="x10", fig=fig, ax=ax)

They can also be added and subtracted, which is useful for combining spectra.

In [ ]:
double_sed = sed + sed

fig, ax = double_sed.plot_spectra(label="double")
fig, ax = sed.plot_spectra(label="original", fig=fig, ax=ax)

## Computing derived properties

There are numerous methods for computing derived properties from an ``Sed`` object. Here we will cover these methods.

### Calculating the bolometric luminosity

We can calculate the bolometric luminosity of the Sed using the `bolometric_luminosity` property method. 

In [ ]:
sed.bolometric_luminosity

This property integrates along the wavelength axis regardless of the shape of the ``Sed``. 

By default this will use a trapezoid integration method. To use the Simpson's rule we can use the explicit `measure_bolometric_luminosity` method.

In [ ]:
sed.measure_bolometric_luminosity(integration_method="simps")

### Computing luminosities in a window

You can also get the ``luminosity`` or ``lnu`` in a particular window by passing the wavelengths defining the limits of the window.

In [ ]:
sed.measure_window_luminosity((1400.0 * Angstrom, 1600.0 * Angstrom))

In [ ]:
sed.measure_window_lnu((1400.0 * Angstrom, 1600.0 * Angstrom))

Notice how units were provided with this window. These are required but also allow you to use an arbitrary unit system.

In [ ]:
sed.measure_window_luminosity((0.14 * um, 0.16 * um))

As for the bolometric luminosity, there are multiple integration methods that can be used for calculating the window.
By default synthesizer will use the trapezoid method (`"trapz"`), but you can also take a simple average.

In [ ]:
sed.measure_window_lnu(
    (1400.0 * Angstrom, 1600.0 * Angstrom),
    integration_method="average",
)

### Measuring breaks 

You can also measure spectral breaks, such as the Balmer break, using the `measure_break` method and providing the wavelengths defining the limits of the break.

In [ ]:
sed.measure_break((3400, 3600) * Angstrom, (4150, 4250) * Angstrom)

For the Balmer break specifically, you can use the `measure_balmer_break` method, which will automatically use the correct wavelengths for the break.

In [ ]:
sed.measure_balmer_break()

### Measuring line indices and spectral slopes

We can also measure absorption line indices, and spectral slopes (e.g. the UV spectral slope $\beta$).

Like the breaks we can measure an arbitrary line index by providing the wavelengths defining the limits of the windows used to measure the index.

In [ ]:
sed.measure_index(
    (1500, 1600) * Angstrom,
    (1400, 1500) * Angstrom,
    (1600, 1700) * Angstrom,
)

Or we can use specialised methods like `measure_d4000()` or `measure_beta()` for the UV spectral slope.


In [ ]:
sed.measure_d4000()

In [ ]:
sed.measure_beta()

By default this uses a single window and fits the spectrum by a power-law. However, we can also specify two windows as below, in which case the luminosity in each window is calculated and used to infer a slope.

In [ ]:
sed.measure_beta(window=(1250, 1750, 2250, 2750) * Angstrom)

### Computing the ionising photon production rate

We can also measure ionising photon production rates at a particular ionisation energy.

In [ ]:
sed.calculate_ionising_photon_production_rate(
    ionisation_energy=13.6 * eV,
    limit=1000,
)

## Observed frame SED

Everything up until now has been in the rest frame, with luminosity densities in units of erg/s/Hz or erg/s/Angstrom. A ``Sed`` object primarily stores the rest frame SED, and treats the observer frame SED as a derived property. 

To move a ``Sed`` to the observer frame we simply call ``get_fnu`` with a cosmology, using an `astropy.cosmology` object, a redshift $z$, and optionally an IGM absorption model (see [here](../../emission_models/attenuation/igm.ipynb) for details).

In [ ]:
from astropy.cosmology import Planck18 as cosmo

from synthesizer.emission_models.attenuation import Madau96

sed.get_fnu(cosmo, z=3.0, igm=Madau96)

fig, ax = sed.plot_observed_spectra(ylimits=(1, 10**3.2))

The result observer frame wavelenths and fluxes are stored in ``obslam`` and ``fnu`` respectively.

In [ ]:
print(sed)

As you can see in the above print out, a number of other flux/observer frame attributes have now been added.

We can also derive the flux at a distance of 10 pc, using the `get_fnu0` method. This is also the fallback used when ``get_fnu`` is called with a redshift of 0.

In [ ]:
sed.get_fnu0()

sed.plot_observed_spectra()

## Photometry

If we want to derive photometry from a Sed object, we first need a ``FilterCollection`` (or ``Instrument`` containg a ``FilterCollection``) to use. For more details on these see the [Instruments and Filters documentation](../../observatories/observatories.rst) section of the documentation.

Now that we have the observer frame ``Sed`` we can use the ``get_photo_fnu`` method to calculate observed photometry. For rest frame photometry, we would instead use the ``get_photo_lnu`` method.

First lets get a JWST filter collection. 

In [ ]:
from synthesizer.instruments import JWSTNIRCamWide

filters = JWSTNIRCamWide().filters

To get the photometry all we need to do is pass this filter set to the ``get_photo_fnu`` method.

In [ ]:
# Measure observed photometry
fluxes = sed.get_photo_fnu(filters)
print(fluxes)

This returns a ``PhotometryCollection`` object, for more details on these see the [Photometry documentation](../../observables/photometry/photometry.rst).

## Multidimensional SEDs

An ``Sed`` object can hold an arbitrarily shaped spectra array (as long as the final axis is the wavelength axis). One very common instance of this across Synthesizer is where you have spectra for particles making up a simulated galaxy. Below we will make a simple 2 particle example with 2 spectra.

In [ ]:
sed2 = Sed(sed.lam, np.array([sed.lnu, sed.lnu * 2]) * erg / s / Hz)

You can print an ``Sed`` object's shape as if it were a numpy array.

In [ ]:
print(sed2.shape)

And all the methods we have seen so far will work on this multidimensional ``Sed`` object, as you'd expect.

In [ ]:
print(sed2.measure_window_lnu((1400, 1600) * Angstrom))
print(sed2.measure_beta())
print(sed2.measure_beta(window=(1250, 1750, 2250, 2750) * Angstrom))
print(sed2.measure_balmer_break())
print(
    sed2.measure_index(
        (1500, 1600) * Angstrom,
        (1400, 1500) * Angstrom,
        (1600, 1700) * Angstrom,
    )
)
print(
    sed2.calculate_ionising_photon_production_rate(
        ionisation_energy=13.6 * eV, limit=1000
    )
)

And even larger spectra. For example, here we convert an entire grid into a set of spectra and can compute properties just as if it were a single spectrum.

In [ ]:
sed3 = grid.get_sed(spectra_type="incident")
print(sed3.shape)

window = sed3.measure_window_lnu((1400, 1600) * Angstrom)

phot_rate = sed3.calculate_ionising_photon_production_rate(
    ionisation_energy=13.6 * eV, limit=1000
)

Notice that the resulting shape has N-1 dimensions because we always act over the final wavelength axis.

In [ ]:
print(phot_rate.shape)

### Integrating Multidimensional SED objects

Multidimensional ``Sed`` objects can be integrated to produce a single spectrum by simply calling ``sum``.  

In [ ]:
print(sed3.shape)
int_sed3 = sed3.sum()
print(int_sed3.shape)

## Combining SEDs

``Sed`` objects can be combined either via concatenation, to produce a single ``Sed`` holding multiple spectra, or by addition, to add the spectra contained in the input ``Sed`` objects (as shown above). 

To concatenate spectra we use ``Sed.concat()``. 
`Sed.concat` can take an arbitrary number of `Sed` objects to combine.

In [ ]:
print("Shapes before:", sed._lnu.shape, sed2._lnu.shape)
sed3 = sed2.concat(sed)
print("Combined shape:", sed3._lnu.shape)

sed4 = sed2.concat(sed, sed2, sed3)
print("Combined shape:", sed4._lnu.shape)

## Resampling SEDs

An ``Sed`` object can be resampled using the ``get_resampled_sed`` method which returns a new ``Sed`` object with the same spectra but resampled to a new wavelength array. 

The resampling can either be done by downsampling by a factor, or by interpolation onto a new wavelength array.

In [ ]:
new_lam = np.logspace(3, 5, 100) * Angstrom
sed_factor_resampled = sed.get_resampled_sed(resample_factor=5)
sed_resampled = sed.get_resampled_sed(new_lam=new_lam)

fig, ax = sed.plot_spectra(label="Original")
fig, ax = sed_resampled.plot_spectra(
    label="Resampled (new_lam)", fig=fig, ax=ax
)
fig, ax = sed_factor_resampled.plot_spectra(
    label="Resampled (factor)",
    fig=fig,
    ax=ax,
    ylimits=(1e27, 1e30),
    xlimits=(10**2, 10**3.5),
)

## Applying attenuation

To apply attenuation to an `Sed` you can use the `apply_attenuation` method and pass the optical depth and a dust curve (see [Attenuation](../../emission_models/attenuation/attenuation.rst) for more details on dust curves).

In [ ]:
from synthesizer.emission_models.attenuation import PowerLaw

sed4_att = sed4.apply_attenuation(tau_v=0.7, dust_curve=PowerLaw(-1.0))

# Integrate the multidimensional spectra
int_sed4 = sed4.sum()
int_sed4_att = sed4_att.sum()

fig, ax = int_sed4.plot_spectra(label="Original")
fig, ax = int_sed4_att.plot_spectra(label="Attenuated", fig=fig, ax=ax)

``apply_attenuation`` can also accept a ``mask``, which applies attenuation to specific spectra in a multidimensional ``Sed`` (like an `Sed` containing the spectra for multiple particles, for instance.)

In [ ]:
sed0_att = sed4.apply_attenuation(
    tau_v=0.7,
    dust_curve=PowerLaw(-1.0),
    mask=np.array([0, 1, 0, 0, 0, 0, 0, 0], dtype=bool),
)

plt.plot(np.log10(sed4.lam), np.log10(sed4.lnu[1, :]), label="Incident")
plt.plot(np.log10(sed4.lam), np.log10(sed0_att.lnu[0, :]), label="Attenuated")
plt.xlim(2.6, 3.5)
plt.ylim(26.0, 30.0)
plt.xlabel(r"$\rm log_{10}(\lambda/\AA)$")
plt.ylabel(r"$\rm log_{10}(L_{\nu}/erg\ s^{-1}\ Hz^{-1} M_{\odot}^{-1})$")
plt.legend()
plt.show()
plt.close()

## Calculating transmission

If you have an attenuated SED, a natural quantity to calculate is the extinction of that spectra ($A$). This can be done either at the wavelengths of the `Sed`, an arbitrary wavelength/wavelength array, or at commonly used values (1500 and 5500 angstrom) using functions available in the `emissions.sed` submodule. Note that these functions return the extinction/attenuation in magnitudes. Below is a demonstration.

In [ ]:
from unyt import angstrom, um

from synthesizer.emissions import (
    get_attenuation,
    get_attenuation_at_1500,
    get_attenuation_at_5500,
    get_attenuation_at_lam,
)

# Get an intrinsic spectra
grid_point = grid.get_grid_point(log10ages=7, metallicity=0.01)
int_sed = grid.get_sed_at_grid_point(grid_point, spectra_type="incident")

# Apply an attenuation curve
att_sed = int_sed.apply_attenuation(tau_v=0.7, dust_curve=PowerLaw(-1.0))

# Get attenuation at sed.lam
attenuation = get_attenuation(int_sed, att_sed)

# Get attenuation at 5 microns
att_at_5 = get_attenuation_at_lam(5 * um, int_sed, att_sed)

# Get attenuation at an arbitrary range of wavelengths
att_at_range = get_attenuation_at_lam(
    np.linspace(5000, 10000, 5) * angstrom, int_sed, att_sed
)

# Get attenuation at 1500 angstrom
att_at_1500 = get_attenuation_at_1500(int_sed, att_sed)

# Get attenuation at 5500 angstrom
att_at_5500 = get_attenuation_at_5500(int_sed, att_sed)

### Doppler broadening

`Sed` objects also provide tools for producing Doppler-broadened versions of an SED. The method `doppler_broaden` returns a new SED broadened using a specified velocity dispersion. This can either operate in-place or return a new `Sed` object.



In [ ]:
from unyt import km, s

log10age = 6.0  # log10(age/yr)
metallicity = 0.01
spectra_type = "nebular"
grid_point = grid.get_grid_point(log10ages=log10age, metallicity=metallicity)

# Extract SED
sed = grid.get_sed_at_grid_point(grid_point, spectra_type=spectra_type)

# Apply Doppler broadening specifying a velocity
sed_broadened1 = sed.doppler_broaden(1000.0 * km / s)

# Plot to compare
fig, ax = sed.plot_spectra(label="original")
fig, ax = sed_broadened1.plot_spectra(
    label="broadened", fig=fig, ax=ax, xlimits=(1000, 1400)
)
ax.set_xscale("linear")

### Thermal Broadening

There is also a variant of this method for thermal broadening, called `thermally_broaden`, which computes the broadening from a given temperature and mean molecular mass (which defaults to `1*amu`). This can either operate in-place or return a new `Sed` object.

In [ ]:
from unyt import K

sed_thermal = sed.thermally_broaden(1e4 * K)
fig, ax = sed.plot_spectra(label="original")
fig, ax = sed_thermal.plot_spectra(
    label="thermally broadened", fig=fig, ax=ax, xlimits=(1000, 1400)
)
ax.set_xscale("linear")

These methods are particularly useful when modelling environments where thermal motions contribute to line broadening, such as H II regions.

Note that, if you have a particle based component, self-consistent Doppler broadening based on particle velocities can be applied during emission generation by enabling `vel_shift` on an `EmissionModel`.